This notebook is for mass generation of the training data for ANN-1D-PIB. It makes use of the ./solve_poly executable which is a Fortran program to numerically calculate the energies for a 1D particle in a box. 

In [15]:
import subprocess
import random

The genererated data is dictated by two parameters:
- L_max - the total size of the box. Specifically, in the code the size of the box will be defined as from -L/2 to L/2. 
- A - the parameter A gives us the largest absolute value of the polynomial coefficients for the potential. So the data will have polynomials with coefficients from -A to +A. 

In [16]:
L_max = 1
A = 10

We can use another parameter N to choose how many random samples in this range we want to collect for the training set.

In [17]:
N = 100

Running the code block below will randomly generate N training sample sets based on the previously defined parameters.

In [19]:
for i in range(N):
    L = random.uniform(0, L_max)
    
    coeffs = [random.uniform(-A, A) for _ in range(5)]
    
    with open('coefficients.txt', 'w') as f:
        f.write(f"{L}\n")
        for coeff in coeffs:
            f.write(f"{coeff}\n")
    
    subprocess.run(['./solve_poly'])

print(f"Done. Generated {N} training samples!")

Done. Generated 100 training samples!
